In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import cv2
import os
from keras.models import model_from_json
from keras.preprocessing import image

In [2]:
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Activation
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras import layers
from tensorflow import keras

In [3]:
X_train, train_y, X_test, test_y = [], [], [], []

In [4]:
df = pd.read_csv(r'C:\Users\User\Desktop\data\fer2013.csv')

In [5]:
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [6]:
for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
            X_train.append(np.array(val, 'float32'))
            train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
            X_test.append(np.array(val, 'float32'))
            test_y.append(row['emotion'])
    except:
        print(f"Error occured at index :{index} and row:{row}")

In [7]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 50
width, height = 48, 48

In [8]:
X_train = np.array(X_train, 'float32')
train_y = np.array(train_y, 'float32')
X_test = np.array(X_test, 'float32')
test_y = np.array(test_y, 'float32')

In [9]:
from keras.utils.np_utils import to_categorical

In [10]:
train_y=to_categorical(train_y, num_classes=num_labels)
test_y=to_categorical(test_y, num_classes=num_labels)

In [11]:
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

                 
X_train = X_train.reshape(X_train.shape[0],48, 48, 1)
                 
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [12]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 64)        36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 18, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 9, 64)          0

In [13]:
model.compile(loss=categorical_crossentropy,
              optimizer= 'sgd',
              metrics=['accuracy'])

In [14]:
model.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)

Epoch 1/50
449/449 [==============================] - 179s 398ms/step - loss: 1.8521 - accuracy: 0.2351 - val_loss: 1.8176 - val_accuracy: 0.2533
Epoch 2/50
449/449 [==============================] - 163s 364ms/step - loss: 1.7976 - accuracy: 0.2527 - val_loss: 1.7958 - val_accuracy: 0.2547
Epoch 3/50
449/449 [==============================] - 164s 365ms/step - loss: 1.7863 - accuracy: 0.2565 - val_loss: 1.7858 - val_accuracy: 0.2566
Epoch 4/50
449/449 [==============================] - 164s 365ms/step - loss: 1.7727 - accuracy: 0.2586 - val_loss: 1.7776 - val_accuracy: 0.2588
Epoch 5/50
449/449 [==============================] - 164s 364ms/step - loss: 1.7620 - accuracy: 0.2696 - val_loss: 1.7485 - val_accuracy: 0.2842
Epoch 6/50
449/449 [==============================] - 166s 369ms/step - loss: 1.7359 - accuracy: 0.2974 - val_loss: 1.7029 - val_accuracy: 0.3059
Epoch 7/50
449/449 [==============================] - 162s 360ms/step - loss: 1.6959 - accuracy: 0.3204 - val_loss: 1.6679 -

In [15]:
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")

In [1]:
test_image = cv2.imread(r'C:\Users\User\Desktop\data\hap.jpg')
cv2.imshow("Image", test_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [20]:
gray_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
cv2.imshow("Image", gray_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
face_cascade= cv2.CascadeClassifier(r'C:\Users\User\Desktop\data\haarcascade_frontalface_default.xml')
from keras.preprocessing.image import img_to_array
cv2.imshow("Image", test_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [1]:
faces = face_cascade.detectMultiScale(gray_image, 1.1, 4)
for (x, y, w, h) in faces:
    cv2.rectangle(test_image,(x,y),(x+w,y+h), (255,0,0))
    roi_gray=gray_image[y:y+w,x:x+h]
    roi_gray=cv2.resize(roi_gray,(48,48))
    image_pixels =img_to_array(roi_gray)
    image_pixels = np.expand_dims(image_pixels, axis = 0)
    image_pixels /= 255
    predictions = model.predict(image_pixels)
    max_index = np.argmax(predictions[0])
    emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
    emotion_prediction = emotion_detection[max_index]
    print(emotion_prediction)
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (50, 50)
    fontScale = 1
    color = (255, 0, 0)
    thickness = 2
    image = cv2.putText(test_image, emotion_prediction, org, font,
                       fontScale, color, thickness, cv2.LINE_AA)
    cv2.imshow("Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

NameError: name 'face_cascade' is not defined